<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/ARAFS/prep_data/track_buoy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy

In [ ]:
import glob as glob
from datetime import date, timedelta

import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
def load_and_select_data(date):

  plot_date = pd.to_datetime(date)
  pdy = plot_date.strftime('%Y%m%d')
  iquam_buoy_fName = data_path + 'iquam_buoy_' + pdy + '*.nc'

  try:
    buoy_file = glob.glob(iquam_buoy_fName)[0]
    #print(f"Loading buoy data from: {buoy_file}")
    ds_buoy = xr.open_dataset(buoy_file)
    return ds_buoy
  except IndexError:
    print(f"No buoy data found for date: {date}")
    return None, None
  except FileNotFoundError:
    print(f"Buoy file not found for date: {date}")
    return None, None

# Inputs

In [ ]:
data_path = '/content/drive/MyDrive/UFS-no-RTOFS/AR/work/data/'

# ERA5
era5_precip_fName   = data_path + 'era5_total_precip_case1.nc'
era5_sst_mslp_fName = data_path + 'era5_skt_mslp_case1.nc'

# CMS
ostia_sst_fName = data_path + 'SST_case1.nc'
cns_sss_fName   = data_path + 'SSS_case1.nc'
aviso_ssh_fName = data_path + 'aviso_ssh_case1.nc'

In [ ]:
# start and end dates of `case 1`:
start_date, end_date = ['2022-12-20', '2022-12-31']

lon_min, lon_max = [-179, -100] # deg West
lat_min, lat_max = [10, 75] # deg North

# ERA5 accumulated rainfall

In [ ]:
era5_precip = xr.open_dataset(era5_precip_fName)
plot_date = pd.to_datetime(start_date)

while plot_date <= pd.to_datetime(end_date):
  #print(plot_date)
  today_precip = era5_precip.sel(valid_time=plot_date, method='nearest').isel(time=1)

  if plot_date == pd.to_datetime(start_date):
    total_precip = today_precip.tp
  elif plot_date == pd.to_datetime(end_date):
    total_precip = total_precip + today_precip.tp # last day
    #total_precip.plot()
  else:
    precip_today = today_precip.tp
    total_precip += precip_today

  plot_date = plot_date + timedelta(days=1)

# Plot
- ERA5 accumulated precipitation and buoy locations

In [ ]:
plot_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Add coastlines and land features
ax.coastlines(color='black', linewidth=1)
ax.add_feature(cfeature.LAND, edgecolor='black', alpha=0.75)

legend_handles = []
legend_labels = []

while plot_date <= end_date:
    ds_buoy = load_and_select_data(plot_date)
    scatter = ax.scatter(ds_buoy.lon, ds_buoy.lat, s=16, label=plot_date.strftime('%Y-%m-%d'), transform= ccrs.PlateCarree(), zorder=100)
    legend_handles.append(scatter)
    legend_labels.append(plot_date.strftime('%Y-%m-%d'))

    if plot_date == end_date:
      total_precip.plot(ax=ax, vmin=0, vmax=0.01, cmap='Greys', add_colorbar=False, transform=ccrs.PlateCarree(), zorder=0)

    plot_date = plot_date + timedelta(days=1)

# Add gridlines
gl = ax.gridlines(draw_labels=True, zorder=0)
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

# Create a single legend with handles and labels collected in the loop
ax.legend(legend_handles, legend_labels, ncol=1, loc='upper left', bbox_to_anchor=(1, 1), fontsize=8)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.set_title('ERA5 accumulated precipitation and drifting buoy locations')

plt.show()

# Pick a region of interest (ROI) and find how many (unique) buoys are there in that region.

In [ ]:
  lon_roi = [-170, -160]
  lat_roi = [40, 42]

In [ ]:
ds_roi = None

plot_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

while plot_date <= end_date:
  print(plot_date)
  ds_buoy = load_and_select_data(plot_date)

  ds_filter = ds_buoy.where( (ds_buoy.lon >= lon_roi[0]) & (ds_buoy.lon <= lon_roi[1]) & (ds_buoy.lat>= lat_roi[0]) & (ds_buoy.lat<=lat_roi[1]))
  ds_filter = ds_filter.dropna(dim='obs')

  if ds_filter.sizes['obs'] > 0:
    if ds_roi is None:
      ds_roi = ds_filter
    else:
      ds_roi = xr.concat([ds_roi, ds_filter], dim='obs')

  plot_date = plot_date + timedelta(days=1)

if ds_roi is not None:
    print("Concatenated dataset ds_roi created.")
    display(ds_roi)
else:
    print("No data found within the specified region of interest.")

# Print WMO ID of buoy(s) found in the region

In [ ]:
ds_roi.pId.values

# Replot (as above) but only with those buoy(s) in the region of interest.

In [ ]:
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Add coastlines and land features
ax.coastlines(color='black', linewidth=1)
ax.add_feature(cfeature.LAND, edgecolor='black', alpha=0.75)

total_precip.plot(ax=ax, vmin=0, vmax=0.01, cmap='Greys', add_colorbar=False, transform=ccrs.PlateCarree(), zorder=0)

scatter = ax.scatter(ds_roi.lon, ds_roi.lat, s=16, c='r', transform=ccrs.PlateCarree(), zorder=100)

# Add gridlines
gl = ax.gridlines(draw_labels=True, zorder=0)
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

# Create a single legend with handles and labels collected in the loop
ax.legend(legend_handles, legend_labels, ncol=1, loc='upper left', bbox_to_anchor=(1, 1), fontsize=8)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.set_title('ERA5 accumulated precipitation and drifting buoy locations')

plt.show()

# Plot time-series of:
- SST (buoy).
- Precipitation (ERA5).

In [ ]:
# ERA5 SST (OSTIA SST) and MSLP
era5_sst_mslp = xr.open_dataset(era5_sst_mslp_fName).isel(time=slice(1, None))

In [ ]:
#print(ds_roi.pId.values)
pidx = '2101829'
ds_roi.to_netcdf(f'buoy_WMOid_{pidx}.nc')

# Create a time series plot of SST from ds_roi and add a second y-axis for precipitation

# Create datetime objects from the year, month, day, hour, and minute
times = pd.to_datetime({
    'year': ds_roi['year'].values.astype(int),
    'month': ds_roi['month'].values.astype(int),
    'day': ds_roi['day'].values.astype(int),
    'hour': ds_roi['hour'].values.astype(int),
    'minute': ds_roi['minute'].values.astype(int)
})

era5_rain_buoy_loc = np.zeros(len(times))
era5_sst_buoy_loc = np.zeros(len(times))

# ERA5 precipitation and SST (not SKT) at buoy location
for iT, obs_t in enumerate(times):
  #print(obs_t.strftime('%Y-%m-%d %H:%M'))
  x1=era5_precip.sel(valid_time=obs_t.strftime('%Y-%m-%d %H:%M'), method='nearest').isel(time=1).sel(latitude=slice(lat_roi[0], lat_roi[1]), longitude=slice(lon_roi[0], lon_roi[1]))
  era5_rain_buoy_loc[iT] = x1.tp.mean(dim=('latitude', 'longitude'))

  y1 = era5_sst_mslp.sel(valid_time=obs_t.strftime('%Y-%m-%d %H:%M'), method='nearest').sel(latitude=slice(lat_roi[0], lat_roi[1]), longitude=slice(lon_roi[0], lon_roi[1]))
  era5_sst_buoy_loc[iT] = y1.sst.mean(dim=('latitude', 'longitude'))

fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.plot(times, ds_roi.sst - 273.15, linestyle='-', c='b', marker='None', label='SST')
ax1.plot(times, era5_sst_buoy_loc-273.15, linestyle='-', c='k', marker='x', label='ERA5 SST')
ax1.set_ylabel('SST ($^\circ$C)', color='b')
ax1.set_xlabel('Date')
ax1.grid(True)
ax1.tick_params(axis='y', colors='b')

# Create a second y-axis
ax2 = ax1.twinx()

# Plot the hurricane track pressure on the second y-axis
ax2.plot(times, era5_rain_buoy_loc, linestyle='-', color='k', marker='x', label='rain')
ax2.set_ylabel('ERA5 total precip (m)', color='k')
ax2.tick_params(axis='y', colors='k')

# Add legends for both axes
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax1.set_title(f'WMO ID {pidx}')